Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [70]:
from random import choices, randint
from dataclasses import dataclass

import lab9_lib

In [71]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

01001111010110010101010110000110100010001011000001: 7.34%
01101110100010100000111001001111111110011000000100: 7.33%
00111111101101101011001111011110111111010111011100: 9.11%
11010101000110000111000111010011000001111111101000: 15.34%
00101110100011101111101110011000100110110011001111: 19.13%
11001000100011001000111110011111101010011011010110: 15.33%
01001110011001111111100111101010000100000110000111: 7.33%
00010010111011010100001111010110011010101000100110: 7.33%
01111101100001110111000011010011011110101100100000: 9.13%
00001001000101000000111010100011000101101111100111: 7.33%
10


A class to describe a single element

In [72]:
@dataclass
class Element:
    def __init__(self, gen) -> None:
        self.gen = gen
        self.fit = fitness(gen)

    def update_fit(self, fit):
        self.fit = fit

Some constants

In [73]:
GENOME_LENGTH = 1_000
TOURNAMENT_SIZE = 3
NUM_EPOCH = 100
OFFSPRING_SIZE = STARTING_SIZE = 1000
PROBLEM_SIZE = 1

A class to contains all the population

In [74]:
@dataclass
class Population:
    def __init__(self, size_starting_population: int):
        self.population = []
        for _ in range(size_starting_population):
            gen = choices([0, 1], k=GENOME_LENGTH)  # K = 1000
            self.population.append(Element(gen))

    def add_offspring(self, offspring):
        self.population.append(offspring)

    def scale_down_population(self, size):
        self.population.sort(key=lambda e: e.fit, reverse=True)
        self.population = self.population[:size]

    def pickone_random(self) -> Element:
        rand_int = randint(0, len(self.population) - 1)
        return self.population[rand_int]

Crossover and mutation
I play a 3 people tournament to select the parents and than I perform a little mutation on the offspring

In [75]:
def tournament(population: Population, tournament_size=1) -> Element:
    best_elem = None
    for _ in range(tournament_size):
        elem = population.pickone_random()
        if best_elem == None:
            best_elem = elem
        if best_elem.fit < elem.fit:
            best_elem = elem
    return best_elem


def mutate(element: Element, num_of_changes=1) -> Element:
    """I've implemented the possiibilty to mutete more than one bit"""
    for _ in range(num_of_changes):
        rand_index = randint(0, GENOME_LENGTH-1)
        element.gen[rand_index] = 1 - element.gen[rand_index]
    return element

def one_cut_crossover(parents) -> Element:
    """function to do one cut crossover"""
    rand_int = randint(0, GENOME_LENGTH-1)
    new_gen = parents[0].gen[rand_int:] + parents[1].gen[:rand_int]
    return Element(new_gen)

def crossover(parents) -> Element:
    """classic crossover"""
    new_gen = []
    for i in range(GENOME_LENGTH):
        new_gen.append(parents[i%2].gen[i])
    return Element(new_gen)

In [76]:
# define the population
population = Population(STARTING_SIZE)

# start the problem
fitness = lab9_lib.make_problem(PROBLEM_SIZE)

for _ in range(NUM_EPOCH):
    for _ in range(OFFSPRING_SIZE):
        parent1 = tournament(population, TOURNAMENT_SIZE)
        parent2 = tournament(population, TOURNAMENT_SIZE)
        elem = one_cut_crossover([parent1,parent2])
        # let's add mutation
        elem = mutate(elem, 2)
        population.add_offspring(elem)
    population.scale_down_population(OFFSPRING_SIZE)
    if population.population[0].fit == 1:
        break


population.scale_down_population(1)
print(f"The best result found is : {population.population.pop().fit} with {fitness.calls} fitness calls")

The best result found is : 1.0 with 82000 fitness calls


In [77]:
#notebook cell just used to test functions
population = Population(3)
elem = population.population.pop()
print(elem.gen)
elem = mutate(elem)
print(elem.gen)

[1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 